In [1]:
import os
from PIL import Image
import numpy as np

openslide_path = r'C:\Users\Donald\Documents\openslide-win64-20230414\bin'
# openslide_path = r'C:\Users\Andre\Documents\openslide-win64-20230414\bin'

os.environ['PATH'] = openslide_path + ";" + os.environ['PATH']
# from openslide import OpenSlide


if hasattr(os, 'add_dll_directory'):
    # Python >= 3.8 on Windows
    with os.add_dll_directory(openslide_path):
        import openslide
else:
    import openslide
from openslide import OpenSlide

You need to do get crop data before running this code. I use a matlab function I made with the 2.5x or 10x tiff images.

Function is here: \\10.99.68.178\andreex\students\Donald Monkey fetus\codes\monkey\crop1024_monkey.m

In [2]:
pth = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\NDPI\pdac purity'
pth_mat = r'\\10.162.80.16\Andre_expansion\data\PDAC purity\old technical assessment\new_tif_images\2_5x\cropped_tile\matfiles'

In [7]:
# Get a list of MAT files and their base names (without extensions)
mat_files = [os.path.splitext(file)[0] for file in os.listdir(pth_mat) if file.endswith('.mat')]
slide_names = [file.split('.')[0] for file in mat_files]

# Get a list of NDPI images and their base names (without extensions)
ndpi_files = [os.path.splitext(file)[0] for file in os.listdir(pth) if file.endswith('.ndpi')]

# Find the matching files based on their base names
matching_files = set(slide_names).intersection(ndpi_files)

WSIs = sorted([filename + '.ndpi' for filename in matching_files])

crop_mats = sorted([filename + '.mat' for filename in mat_files])

In [8]:
crop_mats

['PD_14_1F.mat',
 'PD_23_4O.mat',
 'PD_29_3M.mat',
 'PD_30_3D.mat',
 'PD_34_3L.mat',
 'PD_36_5O.mat',
 'PD_38_9R.mat',
 'PD_44_7HH.mat',
 'PD_45_4M.mat',
 'PD_6_3P_AMP.mat']

In [9]:
WSIs

['PD_14_1F.ndpi',
 'PD_23_4O.ndpi',
 'PD_29_3M.ndpi',
 'PD_30_3D.ndpi',
 'PD_34_3L.ndpi',
 'PD_36_5O.ndpi',
 'PD_38_9R.ndpi',
 'PD_44_7HH.ndpi',
 'PD_45_4M.ndpi',
 'PD_6_3P_AMP.ndpi']

In [10]:
len(WSIs)

10

In [11]:
res20x=0.4416
res2_5x=4

ratio = res2_5x/res20x


outpth = r'\\10.99.68.178\andreex\data\Stardist\PDAC model\maybe_all_tiles_donald\testing_tiles_2_26'
if not os.path.exists(outpth):
    os.mkdir(outpth)

In [12]:
print(ratio)

9.057971014492754


In [13]:
crop_mats

['PD_14_1F.mat',
 'PD_23_4O.mat',
 'PD_29_3M.mat',
 'PD_30_3D.mat',
 'PD_34_3L.mat',
 'PD_36_5O.mat',
 'PD_38_9R.mat',
 'PD_44_7HH.mat',
 'PD_45_4M.mat',
 'PD_6_3P_AMP.mat']

In [14]:
WSIs

['PD_14_1F.ndpi',
 'PD_23_4O.ndpi',
 'PD_29_3M.ndpi',
 'PD_30_3D.ndpi',
 'PD_34_3L.ndpi',
 'PD_36_5O.ndpi',
 'PD_38_9R.ndpi',
 'PD_44_7HH.ndpi',
 'PD_45_4M.ndpi',
 'PD_6_3P_AMP.ndpi']

In [17]:
from scipy.io import loadmat

file_format = '.tif'

for i, file in enumerate(crop_mats):
    mat_file_name = os.path.join(pth_mat, file)

    WSI_path = os.path.join(pth, WSIs[i])

    f_out_name = f'{os.path.join(os.path.join(outpth,crop_mats[i][:-4]))}{file_format}'

    if os.path.exists(f_out_name):
        print(f'Skipping cropping for {WSIs[i]} ({i+1}/{len(crop_mats)})... Already exists')

    else:
        print(f'Cropping {WSIs[i]} ({i+1}/{len(crop_mats)})')

        # print(mat_file_name)
        # print(WSI_path)

        WSI_slide = openslide.OpenSlide(WSI_path)

        wsi_width, wsi_height = WSI_slide.level_dimensions[0]

        data = loadmat(mat_file_name)

        # crop_width = int(data['crop_width']*ratio)
        # crop_height = int(data['crop_height']*ratio)


        crop_width = 512
        crop_height = 512
        crop_x = int(data['crop_x']*ratio)
        crop_y = int(data['crop_y']*ratio)

        lower_x = crop_x
        upper_y = crop_y
        upper_x = crop_x + crop_width
        lower_y = crop_y - crop_height

        print(crop_x)
        print(crop_y)
        print(crop_width)
        print(crop_height)

        region = WSI_slide.read_region((crop_x, crop_y), 0, (crop_width, crop_height)) # level = 0 in middle
        region_image = Image.new('RGB', region.size)
        region_image.paste(region, (0, 0))

        region_image.save(f_out_name)
        WSI_slide.close()


Skipping cropping for PD_14_1F.ndpi (1/10)... Already exists
Skipping cropping for PD_23_4O.ndpi (2/10)... Already exists
Skipping cropping for PD_29_3M.ndpi (3/10)... Already exists
Skipping cropping for PD_30_3D.ndpi (4/10)... Already exists
Skipping cropping for PD_34_3L.ndpi (5/10)... Already exists
Skipping cropping for PD_36_5O.ndpi (6/10)... Already exists
Skipping cropping for PD_38_9R.ndpi (7/10)... Already exists
Cropping PD_44_7HH.ndpi (8/10)
31042
15937
512
512
Cropping PD_45_4M.ndpi (9/10)
46990
37557
512
512
Skipping cropping for PD_6_3P_AMP.ndpi (10/10)... Already exists


In [38]:
crop_x


19818

In [39]:
crop_y

20134